In [2]:
from sklearn.neighbors import KNeighborsRegressor
from sklearn.model_selection import GridSearchCV

param_grid_knn = {
    'n_neighbors': [3, 5, 7, 9, 11, 15, 21],
    'weights': ['uniform', 'distance'],
    'metric': ['euclidean', 'manhattan', 'minkowski'],
    'p': [1, 2]  
}

grid_knn = GridSearchCV(
    KNeighborsRegressor(),
    param_grid_knn,
    cv=5,
    n_jobs=-1
)
grid_knn.fit(X_train_scaled, y_train)
knn_preds = grid_knn.predict(X_test_scaled)
knn_r2 = r2_score(y_test, knn_preds)
knn_mae = mean_absolute_error(y_test, knn_preds)
knn_rmse = np.sqrt(np.mean((y_test - knn_preds)**2))
knn_results = pd.DataFrame({
    "Metric": ["R²", "MAE", "RMSE"],
    "KNN": [knn_r2, knn_mae, knn_rmse]
})
print("Best KNN Params:", grid_knn.best_params_)
display(knn_results)


Best KNN Params: {'metric': 'manhattan', 'n_neighbors': 15, 'p': 1, 'weights': 'distance'}


,Metric,KNN
0,R²,0.784955
1,MAE,0.001601
2,RMSE,0.005770


In [3]:
from sklearn.linear_model import ElasticNet
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_absolute_error
import pandas as pd
import numpy as np

# Improved parameter grid for ElasticNet
param_grid_en = {
    'alpha': [0.001, 0.01, 0.1, 1.0, 10.0],      # Expanded range of regularization
    'l1_ratio': [0.1, 0.3, 0.5, 0.7, 0.9],       # More granular L1/L2 mix
    'max_iter': [10000, 20000],                  # Try different max iterations
    'tol': [1e-4, 1e-3, 1e-2]                    # Tolerance for optimization
}

grid_en = GridSearchCV(
    ElasticNet(random_state=42),
    param_grid_en,
    cv=5,                # Increased CV folds for better validation
    n_jobs=-1
)
grid_en.fit(X_train_scaled, y_train)
en_preds = grid_en.predict(X_test_scaled)
en_r2 = r2_score(y_test, en_preds)
en_mae = mean_absolute_error(y_test, en_preds)
en_rmse = np.sqrt(np.mean((y_test - en_preds)**2))
en_results = pd.DataFrame({
    "Metric": ["R²", "MAE", "RMSE"],
    "ElasticNet": [en_r2, en_mae, en_rmse]
})

print("Best ElasticNet Params:", grid_en.best_params_)
display(en_results)


Best ElasticNet Params: {'alpha': 0.001, 'l1_ratio': 0.1, 'max_iter': 10000, 'tol': 0.0001}


,Metric,ElasticNet
0,R²,0.310706
1,MAE,0.004038
2,RMSE,0.010331


In [5]:
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import GridSearchCV
from sklearn.metrics import r2_score, mean_absolute_error
import pandas as pd
import numpy as np

# Expanded parameter grid for Random Forest
param_grid_rf = {
    'n_estimators': [100, 200, 300],            # Number of trees
    'max_depth': [None, 10, 20, 30],            # Tree depth
    'min_samples_split': [2, 5, 10],            # Minimum samples to split a node
    'min_samples_leaf': [1, 2, 4],              # Minimum samples at a leaf node
    'max_features': ['auto', 'sqrt', 'log2']    # Number of features to consider at each split
}

grid_rf = GridSearchCV(
    RandomForestRegressor(random_state=42, n_jobs=-1), 
    param_grid_rf, 
    cv=5,                # More robust cross-validation
    n_jobs=-1,
    verbose=3
)
grid_rf.fit(X_train_scaled, y_train)
rf_preds = grid_rf.predict(X_test_scaled)
rf_r2 = r2_score(y_test, rf_preds)
rf_mae = mean_absolute_error(y_test, rf_preds)
rf_rmse = np.sqrt(np.mean((y_test - rf_preds)**2))
rf_results = pd.DataFrame({
    "Metric": ["R²", "MAE", "RMSE"],
    "Random Forest": [rf_r2, rf_mae, rf_rmse]
})

print("Best RF Params:", grid_rf.best_params_)
display(rf_results)


Fitting 5 folds for each of 324 candidates, totalling 1620 fits


/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors

[CV 4/5] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=100;, score=0.803 total time=  17.0s
[CV 3/5] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=200;, score=0.975 total time=  33.5s
[CV 2/5] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=2, n_estimators=300;, score=0.994 total time=  49.8s
[CV 1/5] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=100;, score=0.987 total time=  16.0s
[CV 2/5] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=100;, score=0.994 total time=  15.0s
[CV 4/5] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=100;, score=0.793 total time=  14.7s
[CV 2/5] END max_depth=None, max_features=auto, min_samples_leaf=1, min_samples_split=5, n_estimators=200;, score=0.993 total time=  30.2s
[CV 1/5] END max_depth=None

/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors


[CV 1/5] END max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=200;, score=0.932 total time=  11.8s
[CV 5/5] END max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=200;, score=0.777 total time=  11.9s
[CV 4/5] END max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=2, n_estimators=300;, score=0.741 total time=  17.7s
[CV 5/5] END max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=100;, score=0.770 total time=   6.0s
[CV 3/5] END max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=200;, score=0.910 total time=  12.3s
[CV 2/5] END max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=5, n_estimators=300;, score=0.948 total time=  17.6s
[CV 1/5] END max_depth=None, max_features=sqrt, min_samples_leaf=4, min_samples_split=10, n_estimators=100;, score=0.924 total time=   6.2s
[CV 3/5] END max_depth=No

...

/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(
/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors


[CV 2/5] END max_depth=20, max_features=log2, min_samples_leaf=4, min_samples_split=2, n_estimators=300;, score=0.947 total time=  18.0s
[CV 1/5] END max_depth=20, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=100;, score=0.934 total time=   5.9s
[CV 3/5] END max_depth=20, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=100;, score=0.907 total time=   5.8s
[CV 1/5] END max_depth=20, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=200;, score=0.933 total time=  11.7s
[CV 5/5] END max_depth=20, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=200;, score=0.772 total time=  12.0s
[CV 4/5] END max_depth=20, max_features=log2, min_samples_leaf=4, min_samples_split=5, n_estimators=300;, score=0.740 total time=  17.4s
[CV 5/5] END max_depth=20, max_features=log2, min_samples_leaf=4, min_samples_split=10, n_estimators=100;, score=0.774 total time=   6.5s
[CV 3/5] END max_depth=20, max_features

/usr/local/lib/python3.11/dist-packages/sklearn/ensemble/_forest.py:413: FutureWarning: `max_features='auto'` has been deprecated in 1.1 and will be removed in 1.3. To keep the past behaviour, explicitly set `max_features=1.0` or remove this parameter as it is also the default value for RandomForestRegressors and ExtraTreesRegressors.
  warn(


Best RF Params: {'max_depth': None, 'max_features': 'auto', 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 200}


,Metric,Random Forest
0,R²,0.948081
1,MAE,0.000190
2,RMSE,0.002835
